After having trained a model, you can use the output.yml and an input sequence file to make predictions

## IMPORTANT BEFORE YOU START

The out config will contain absolute paths to directories. In other words: this notebook will likely work if you ran the training and notebook from the same environment (node, cluster or machine), but it will most likely fail to work if you move results from training to a different machine to perform inference/predictions. However, don't worry: there's an easy fix (see below).

In [1]:
from biotrainer.utilities import read_config_file
from biotrainer.inference import Inferencer

In [2]:
out_config_path = '../residue_to_class/output/out.yml'
out_config = read_config_file(out_config_path)

Let's find out how well the model performs on the test set.

In [3]:
print(f"For the {out_config['model_choice']}, the metrics on the test set are:")
for metric in out_config['test_iterations_results']['metrics']:
    print(f"\t{metric} : {out_config['test_iterations_results']['metrics'][metric]}")

For the CNN, the metrics on the test set are:
	- f1_score class 0 : 0.0
	- f1_score class 1 : 0.0
	- f1_score class 2 : 0.0
	- f1_score class 3 : 0.0
	- f1_score class 4 : 0.0
	- precission class 0 : 0.0
	- precission class 1 : 0.0
	- precission class 2 : 0.0
	- precission class 3 : 0.0
	- precission class 4 : 0.0
	- recall class 0 : 0.0
	- recall class 1 : 0.0
	- recall class 2 : 0.0
	- recall class 3 : 0.0
	- recall class 4 : 0.0
	accuracy : 0.0
	loss : 1.6182039976119995
	macro-f1_score : 0.0
	macro-precision : 0.0
	macro-recall : 0.0
	matthews-corr-coeff : -0.3000600337982178
	micro-f1_score : 0.0
	micro-precision : 0.0
	micro-recall : 0.0
	spearmans-corr-coeff : -0.14046210050582886


**Does the absolute path of the model look correct?**

As stated above, the out.yml file will contain absolute paths to files and directories from the biotrainer run. If you move files between machines, these paths may get "broken". However, in order to fix this, you juse need to substitute the beginning of the path as stored in the outconfig with the location of where the results are stored now. This is fairly easy, an example is provided below, but needs to be adapted to **your local folder structure**!



In [4]:
print(f"Absolute path of biotrainer run output as from config: {out_config['output_dir']}")

Absolute path of biotrainer run output as from config: /home/sebie/PycharmProjects/biotrainerFork/examples/residue_to_class/output


In [6]:
new_output_path_root = "../examples/"
old_output_path_root = "/mnt/home/cdallago/biotrainer/examples/"

for key in out_config:
    if isinstance(out_config[key], str) and old_output_path_root in out_config[key]:
        out_config[key] = out_config[key].replace(old_output_path_root, new_output_path_root)

In [7]:
print(f"Absolute path of biotrainer run output after swapping the root path: {out_config['output_dir']}")

Absolute path of biotrainer run output after swapping the root path: /home/sebie/PycharmProjects/biotrainerFork/examples/residue_to_class/output


First we need to create the embeddings for the sequences we are interested in

In [8]:
from bio_embeddings.embed import OneHotEncodingEmbedder

In [9]:
embedder = OneHotEncodingEmbedder()

In [10]:
sequences = [
    "PROVTEIN",
    "SEQVENCESEQVENCE"
]

In [11]:
embeddings = embedder.embed_many(sequences)
# Note that for per-sequence embeddings, you would have to reduce the embeddings now:
# embeddings = [[embedder.reduce_per_protein(embedding)] for embedding in embeddings]

Next we generate an inference object from the out config of our training run

In [12]:
inferencer = Inferencer(**out_config)

Got 1 split(s): hold_out


/home/sebie/.cache/pypoetry/virtualenvs/biotrainer-_BxfB8Sv-py3.9/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [15]:
predictions = inferencer.from_embeddings(embeddings, split_name="hold_out")

We can inspect the predictions

In [16]:
for sequence, prediction in zip(sequences, predictions.values()):
    print(sequence)
    print(prediction)

PROVTEIN
FFFDFDFF
SEQVENCESEQVENCE
FFEFFFFFDEFFFFEF
